In [11]:
import geopandas as gpd
import pandas as pd
from shapely.geometry import Point
%matplotlib inline
import seaborn as sns; sns.set_theme(color_codes=True)
pd.set_option('display.max_columns', None)
import matplotlib.pyplot as plt
import numpy as np
import matplotlib.colors
import warnings
warnings.filterwarnings('ignore')
from scipy import stats
import math    
import folium
from scipy.cluster.hierarchy import linkage, fcluster
from scipy.spatial.distance import pdist
import sys
from haversine import haversine
import matplotlib.image as mpimg
import glob, os
from pathlib import Path
from IPython.core.display import display,HTML
pd.set_option('display.max_colwidth', None)

In [12]:
from src.cluster import *
from src.colors import *
from src.map import *
from src.compare import *

In [13]:
labels = pd.read_csv('datasets/mikey-devon-labels_validation-221126.csv')
labels

,Unnamed: 0,username,label_id,street_edge_id,neighborhood,label_type,lat,lng,gsv_panorama_id,time_created,correct,severity,temporary,tag_list,description
0,3,Devon Snyder,3,480,University District,Obstacle,47.661041,-122.320709,-L8Sgb0ozIqqanCBfAZjYQ,2022-05-10 14:00:36.787-07,t,3.0,f,pole,NaN
1,4,Devon Snyder,4,480,University District,Obstacle,47.661026,-122.320702,-L8Sgb0ozIqqanCBfAZjYQ,2022-05-10 14:00:52.367-07,t,2.0,f,pole,NaN
2,5,Devon Snyder,5,480,University District,SurfaceProblem,47.661060,-122.320709,OH4yKYW98UY1aMK_b68Etw,2022-05-10 14:01:21.608-07,t,2.0,f,bumpy,NaN
3,8,Devon Snyder,8,480,University District,Occlusion,47.661106,-122.320724,OH4yKYW98UY1aMK_b68Etw,2022-05-10 14:02:15.133-07,NaN,NaN,f,NaN,NaN
4,9,Devon Snyder,9,480,University District,Obstacle,47.661098,-122.320663,O0VZ-MF6Nrmjhkb2nCRhCw,2022-05-10 14:02:29.636-07,t,3.0,f,sign,temporary due to construction
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2015,2533,Devon Snyder,2533,479,University District,Occlusion,47.659710,-122.319855,bWNcSsunJ_MyQcFwru1edg,2022-11-09 08:08:14.709-08,NaN,NaN,f,NaN,NaN
2016,2534,Devon Snyder,2534,479,University District,SurfaceProblem,47.659721,-122.319923,SRZ-Py452sb8rEJfkHzHWw,2022-11-09 08:08:26.318-08,NaN,2.0,f,"cracks,uneven/slanted",NaN
2017,2535,Devon Snyder,2535,479,University District,Occlusion,47.659653,-122.319908,SRZ-Py452sb8rEJfkHzHWw,2022-11-09 08:08:36.919-08,NaN,NaN,f,NaN,NaN
2018,2536,Devon Snyder,2536,479,University District,SurfaceProblem,47.659668,-122.319916,SRZ-Py452sb8rEJfkHzHWw,2022-11-09 08:08:40.146-08,NaN,2.0,f,cracks,NaN


In [14]:
#create labels geo dataframe
points = labels.apply(lambda row: Point(row.lng, row.lat), axis=1)
labels_geo = gpd.GeoDataFrame(labels, geometry=points)
labels_geo.crs = {'init': 'epsg:4326'}
labels_geo

,Unnamed: 0,username,label_id,street_edge_id,neighborhood,label_type,lat,lng,gsv_panorama_id,time_created,correct,severity,temporary,tag_list,description,geometry
0,3,Devon Snyder,3,480,University District,Obstacle,47.661041,-122.320709,-L8Sgb0ozIqqanCBfAZjYQ,2022-05-10 14:00:36.787-07,t,3.0,f,pole,NaN,POINT (-122.32071 47.66104)
1,4,Devon Snyder,4,480,University District,Obstacle,47.661026,-122.320702,-L8Sgb0ozIqqanCBfAZjYQ,2022-05-10 14:00:52.367-07,t,2.0,f,pole,NaN,POINT (-122.32070 47.66103)
2,5,Devon Snyder,5,480,University District,SurfaceProblem,47.661060,-122.320709,OH4yKYW98UY1aMK_b68Etw,2022-05-10 14:01:21.608-07,t,2.0,f,bumpy,NaN,POINT (-122.32071 47.66106)
3,8,Devon Snyder,8,480,University District,Occlusion,47.661106,-122.320724,OH4yKYW98UY1aMK_b68Etw,2022-05-10 14:02:15.133-07,NaN,NaN,f,NaN,NaN,POINT (-122.32072 47.66111)
4,9,Devon Snyder,9,480,University District,Obstacle,47.661098,-122.320663,O0VZ-MF6Nrmjhkb2nCRhCw,2022-05-10 14:02:29.636-07,t,3.0,f,sign,temporary due to construction,POINT (-122.32066 47.66110)
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2015,2533,Devon Snyder,2533,479,University District,Occlusion,47.659710,-122.319855,bWNcSsunJ_MyQcFwru1edg,2022-11-09 08:08:14.709-08,NaN,NaN,f,NaN,NaN,POINT (-122.31985 47.65971)
2016,2534,Devon Snyder,2534,479,University District,SurfaceProblem,47.659721,-122.319923,SRZ-Py452sb8rEJfkHzHWw,2022-11-09 08:08:26.318-08,NaN,2.0,f,"cracks,uneven/slanted",NaN,POINT (-122.31992 47.65972)
2017,2535,Devon Snyder,2535,479,University District,Occlusion,47.659653,-122.319908,SRZ-Py452sb8rEJfkHzHWw,2022-11-09 08:08:36.919-08,NaN,NaN,f,NaN,NaN,POINT (-122.31991 47.65965)
2018,2536,Devon Snyder,2536,479,University District,SurfaceProblem,47.659668,-122.319916,SRZ-Py452sb8rEJfkHzHWw,2022-11-09 08:08:40.146-08,NaN,2.0,f,cracks,NaN,POINT (-122.31992 47.65967)


In [15]:
# Seperate Seattle and Chicago data
chicago = labels_geo.loc[labels_geo['lng'] > -105]
seattle = labels_geo.loc[labels_geo['lng'] < -105]

In [16]:
#map_clusters(chicago,0)

In [17]:
#get curb ramp cluster dataset for chicago 
chicago_curb_ramp = cluster_label_type_at_index(chicago,0)[2]
#get problem cluster dataset for chicago 
chicago_problem = cluster_label_type_at_index(chicago,2)[2]

In [18]:
# locate folder with images
chicago_folder ="chicago_with_icons/"

In [19]:
def append_img_url(df, folder):
    df['img_url'] = df.apply(lambda x: 'img/'+folder + "label_id_"+str(x.label_id)+".jpg", axis=1)
    return df

In [20]:
append_img_url(chicago_curb_ramp, chicago_folder)

,Unnamed: 0,username,label_id,street_edge_id,neighborhood,label_type,lat,lng,gsv_panorama_id,time_created,correct,severity,temporary,tag_list,description,geometry,cluster_id,img_url
76,94,Devon Snyder,94,447,Skokie,CurbRamp,42.022606,-87.728157,rcjtiNG8gkNtkifZDFj2NQ,2022-05-10 14:56:14.398-07,t,4.0,f,"not enough landing space,narrow,not level with street",NaN,POINT (-87.72816 42.02261),2,img/chicago_with_icons/label_id_94.jpg
77,96,Devon Snyder,96,447,Skokie,CurbRamp,42.022720,-87.728149,rcjtiNG8gkNtkifZDFj2NQ,2022-05-10 14:56:58.603-07,t,3.0,f,"not enough landing space,surface problem",NaN,POINT (-87.72815 42.02272),1,img/chicago_with_icons/label_id_96.jpg
78,97,Devon Snyder,97,437,Skokie,CurbRamp,42.022717,-87.728432,_KrmBjHlebAncoZc40tsSg,2022-05-10 14:57:43.381-07,t,3.0,f,"not enough landing space,points into traffic",NaN,POINT (-87.72843 42.02272),15,img/chicago_with_icons/label_id_97.jpg
79,98,Devon Snyder,98,437,Skokie,CurbRamp,42.022621,-87.728439,_KrmBjHlebAncoZc40tsSg,2022-05-10 14:58:11.132-07,t,3.0,f,not enough landing space,NaN,POINT (-87.72844 42.02262),16,img/chicago_with_icons/label_id_98.jpg
82,102,Devon Snyder,102,447,Skokie,CurbRamp,42.023483,-87.728134,BMhU3MUjwcgFnO2ax-6TxQ,2022-05-10 15:00:54.488-07,t,3.0,f,"surface problem,points into traffic",NaN,POINT (-87.72813 42.02348),3,img/chicago_with_icons/label_id_102.jpg
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1899,2385,mikey,2385,447,Skokie,CurbRamp,42.024353,-87.728111,T14zUxLr9qqMbsGQ_9Jqqg,2022-07-05 12:41:14.317-07,NaN,1.0,f,NaN,NaN,POINT (-87.72811 42.02435),5,img/chicago_with_icons/label_id_2385.jpg
1901,2387,mikey,2387,447,Skokie,CurbRamp,42.023594,-87.728142,tagGeC6m-TMnzcfHLPX6Hg,2022-07-05 12:41:55.266-07,NaN,1.0,f,NaN,NaN,POINT (-87.72814 42.02359),4,img/chicago_with_icons/label_id_2387.jpg
1902,2388,mikey,2388,447,Skokie,CurbRamp,42.023453,-87.728127,TdCToHEwW86oaUfwfNNDaw,2022-07-05 12:42:00.438-07,NaN,1.0,f,NaN,NaN,POINT (-87.72813 42.02345),3,img/chicago_with_icons/label_id_2388.jpg
1903,2389,mikey,2389,447,Skokie,CurbRamp,42.022594,-87.728149,rcjtiNG8gkNtkifZDFj2NQ,2022-07-05 12:42:51.976-07,NaN,1.0,f,NaN,NaN,POINT (-87.72815 42.02259),2,img/chicago_with_icons/label_id_2389.jpg


In [12]:
chicago_curb_ramp

,username,label_id,street_edge_id,neighborhood,label_type,lat,lng,gsv_panorama_id,time_created,correct,severity,temporary,tag_list,description,Unnamed: 14,Unnamed: 15,Unnamed: 16,geometry,cluster_id,img_url
76,Devon Snyder,94,447,Skokie,CurbRamp,42.022606,-87.728157,rcjtiNG8gkNtkifZDFj2NQ,2022-05-10 14:56:14.398-07,t,4.0,f,not level with street,not enough landing space,narrow,NaN,NaN,POINT (-87.72816 42.02261),2,img/chicago_with_icons/label_id_94.jpg
77,Devon Snyder,96,447,Skokie,CurbRamp,42.022720,-87.728149,rcjtiNG8gkNtkifZDFj2NQ,2022-05-10 14:56:58.603-07,t,3.0,f,not enough landing space,surface problem,NaN,NaN,NaN,POINT (-87.72815 42.02272),1,img/chicago_with_icons/label_id_96.jpg
78,Devon Snyder,97,437,Skokie,CurbRamp,42.022717,-87.728432,_KrmBjHlebAncoZc40tsSg,2022-05-10 14:57:43.381-07,t,3.0,f,not enough landing space,points into traffic,NaN,NaN,NaN,POINT (-87.72843 42.02272),15,img/chicago_with_icons/label_id_97.jpg
79,Devon Snyder,98,437,Skokie,CurbRamp,42.022621,-87.728439,_KrmBjHlebAncoZc40tsSg,2022-05-10 14:58:11.132-07,t,3.0,f,not enough landing space,NaN,NaN,NaN,NaN,POINT (-87.72844 42.02262),16,img/chicago_with_icons/label_id_98.jpg
82,Devon Snyder,102,447,Skokie,CurbRamp,42.023483,-87.728134,BMhU3MUjwcgFnO2ax-6TxQ,2022-05-10 15:00:54.488-07,t,3.0,f,surface problem,points into traffic,NaN,NaN,NaN,POINT (-87.72813 42.02348),3,img/chicago_with_icons/label_id_102.jpg
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1914,mikey,2385,447,Skokie,CurbRamp,42.024353,-87.728111,T14zUxLr9qqMbsGQ_9Jqqg,2022-07-05 12:41:14.317-07,NaN,1.0,f,NaN,NaN,NaN,NaN,NaN,POINT (-87.72811 42.02435),5,img/chicago_with_icons/label_id_2385.jpg
1916,mikey,2387,447,Skokie,CurbRamp,42.023594,-87.728142,tagGeC6m-TMnzcfHLPX6Hg,2022-07-05 12:41:55.266-07,NaN,1.0,f,NaN,NaN,NaN,NaN,NaN,POINT (-87.72814 42.02359),4,img/chicago_with_icons/label_id_2387.jpg
1917,mikey,2388,447,Skokie,CurbRamp,42.023453,-87.728127,TdCToHEwW86oaUfwfNNDaw,2022-07-05 12:42:00.438-07,NaN,1.0,f,NaN,NaN,NaN,NaN,NaN,POINT (-87.72813 42.02345),3,img/chicago_with_icons/label_id_2388.jpg
1918,mikey,2389,447,Skokie,CurbRamp,42.022594,-87.728149,rcjtiNG8gkNtkifZDFj2NQ,2022-07-05 12:42:51.976-07,NaN,1.0,f,NaN,NaN,NaN,NaN,NaN,POINT (-87.72815 42.02259),2,img/chicago_with_icons/label_id_2389.jpg


In [24]:
append_img_url(chicago_problem, chicago_folder)

,Unnamed: 0,username,label_id,street_edge_id,neighborhood,label_type,lat,lng,gsv_panorama_id,time_created,correct,severity,temporary,tag_list,description,geometry,cluster_id,img_url
80,99,Devon Snyder,99,447,Skokie,SurfaceProblem,42.022999,-87.728149,2vMz67hXm2ZGNcKHbyoR8g,2022-05-10 14:59:05.467-07,t,1.0,f,grass,NaN,POINT (-87.72815 42.02300),63,img/chicago_with_icons/label_id_99.jpg
81,100,Devon Snyder,100,447,Skokie,SurfaceProblem,42.023033,-87.728149,2vMz67hXm2ZGNcKHbyoR8g,2022-05-10 14:59:10.286-07,t,1.0,f,NaN,NaN,POINT (-87.72815 42.02303),67,img/chicago_with_icons/label_id_100.jpg
84,105,Devon Snyder,105,447,Skokie,SurfaceProblem,42.023663,-87.728111,spnkMkInfnfp6Se2o7-t9Q,2022-05-10 15:02:43.061-07,t,1.0,f,"grass,cracks",NaN,POINT (-87.72811 42.02366),88,img/chicago_with_icons/label_id_105.jpg
85,106,Devon Snyder,106,447,Skokie,SurfaceProblem,42.024178,-87.728104,-YaXYeIUgy5a5bYJcFDMQQ,2022-05-10 15:03:40.542-07,t,1.0,f,"grass,cracks",NaN,POINT (-87.72810 42.02418),1,img/chicago_with_icons/label_id_106.jpg
91,113,Devon Snyder,113,437,Skokie,Obstacle,42.024498,-87.728394,p_-iwPEGdz3zFZzojysq_Q,2022-05-10 15:08:14.409-07,t,4.0,t,construction,NaN,POINT (-87.72839 42.02450),8,img/chicago_with_icons/label_id_113.jpg
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1883,2368,mikey,2368,437,Skokie,SurfaceProblem,42.023743,-87.728401,UUSs_oAVBV5cqnoLKsUnlQ,2022-07-05 12:25:19.838-07,NaN,1.0,f,NaN,NaN,POINT (-87.72840 42.02374),7,img/chicago_with_icons/label_id_2368.jpg
1884,2369,mikey,2369,437,Skokie,SurfaceProblem,42.023582,-87.728416,Ue0KqJfGftuJWhdBCneCkQ,2022-07-05 12:25:31.351-07,NaN,1.0,f,uneven/slanted,NaN,POINT (-87.72842 42.02358),6,img/chicago_with_icons/label_id_2369.jpg
1893,2378,mikey,2378,404,Skokie,SurfaceProblem,42.025799,-87.728088,jfd17OkETDGKKw-I1zK6Jw,2022-07-05 12:38:58.8-07,NaN,1.0,f,bumpy,NaN,POINT (-87.72809 42.02580),5,img/chicago_with_icons/label_id_2378.jpg
1895,2380,mikey,2380,404,Skokie,Obstacle,42.025677,-87.728088,Y-iz2Xxh7q_fAX_UH6mmVA,2022-07-05 12:39:07.658-07,NaN,1.0,t,NaN,NaN,POINT (-87.72809 42.02568),3,img/chicago_with_icons/label_id_2380.jpg


In [15]:
#get center location of the geodf
chicago_curb_ramp.dissolve().centroid

0    POINT (-87.76274 41.93642)
dtype: geometry

In [25]:
#get curb ramp cluster dataset for seattle 
seattle_curb_ramp = cluster_label_type_at_index(seattle,0)[2]
#get problem cluster dataset for seattle 
seattle_problem = cluster_label_type_at_index(seattle,2)[2]

In [26]:
# locate folder with images
seattle_folder ="seattle_with_icons/"

In [27]:
append_img_url(seattle_curb_ramp, seattle_folder)

,Unnamed: 0,username,label_id,street_edge_id,neighborhood,label_type,lat,lng,gsv_panorama_id,time_created,correct,severity,temporary,tag_list,description,geometry,cluster_id,img_url
10,19,Devon Snyder,19,482,University District,CurbRamp,47.661415,-122.320831,j3GefSTHZmrn2LHTGg8SFA,2022-05-10 14:08:11.854-07,t,3.0,f,"narrow,not enough landing space,surface problem",NaN,POINT (-122.32083 47.66142),34,img/seattle_with_icons/label_id_19.jpg
12,21,Devon Snyder,21,482,University District,CurbRamp,47.661472,-122.320847,xx0-vgC0WHpTn2feGbrbUA,2022-05-10 14:09:32.271-07,t,2.0,f,not enough landing space,NaN,POINT (-122.32085 47.66147),33,img/seattle_with_icons/label_id_21.jpg
13,22,Devon Snyder,22,482,University District,CurbRamp,47.661488,-122.320854,qCkbp2lszdGrR5s3hdSiTQ,2022-05-10 14:09:52.251-07,t,2.0,f,"surface problem,not enough landing space",NaN,POINT (-122.32085 47.66149),32,img/seattle_with_icons/label_id_22.jpg
15,27,Devon Snyder,27,483,University District,CurbRamp,47.661472,-122.320702,qCkbp2lszdGrR5s3hdSiTQ,2022-05-10 14:10:59.364-07,t,3.0,f,not level with street,NaN,POINT (-122.32070 47.66147),31,img/seattle_with_icons/label_id_27.jpg
28,41,Devon Snyder,41,486,University District,CurbRamp,47.661228,-122.319885,hr-qZOb3MdbSB1ZNBjsSIw,2022-05-10 14:18:55.439-07,t,2.0,f,surface problem,a bit far away even zoomed in but tried best to rate,POINT (-122.31989 47.66123),41,img/seattle_with_icons/label_id_41.jpg
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2001,2489,mikey,2489,810,Windermere,CurbRamp,47.672092,-122.267960,09WtcNVwH9KSZ8B4Hqm3Bg,2022-07-06 14:12:02.237-07,NaN,2.0,f,surface problem,NaN,POINT (-122.26796 47.67209),5,img/seattle_with_icons/label_id_2489.jpg
2010,2528,Devon Snyder,2528,487,University District,CurbRamp,47.659523,-122.319687,jCGaqx_i5lllqbp3q1GWUw,2022-11-09 08:03:27.714-08,NaN,3.0,f,"missing tactile warning,points into traffic,narrow",NaN,POINT (-122.31969 47.65952),4,img/seattle_with_icons/label_id_2528.jpg
2011,2529,Devon Snyder,2529,488,University District,CurbRamp,47.659401,-122.319725,z9UEjhLFWd_yO6TmMzABfA,2022-11-09 08:04:02.134-08,NaN,2.0,f,"points into traffic,missing tactile warning",NaN,POINT (-122.31973 47.65940),3,img/seattle_with_icons/label_id_2529.jpg
2012,2530,Devon Snyder,2530,476,University District,CurbRamp,47.659454,-122.319939,ndtOrb8y8Cnt1HpvVXWNuQ,2022-11-09 08:04:41.577-08,NaN,2.0,f,"narrow,missing tactile warning",NaN,POINT (-122.31994 47.65945),2,img/seattle_with_icons/label_id_2530.jpg


In [28]:
append_img_url(seattle_problem, seattle_folder)

,Unnamed: 0,username,label_id,street_edge_id,neighborhood,label_type,lat,lng,gsv_panorama_id,time_created,correct,severity,temporary,tag_list,description,geometry,cluster_id,img_url
0,3,Devon Snyder,3,480,University District,Obstacle,47.661041,-122.320709,-L8Sgb0ozIqqanCBfAZjYQ,2022-05-10 14:00:36.787-07,t,3.0,f,pole,NaN,POINT (-122.32071 47.66104),240,img/seattle_with_icons/label_id_3.jpg
1,4,Devon Snyder,4,480,University District,Obstacle,47.661026,-122.320702,-L8Sgb0ozIqqanCBfAZjYQ,2022-05-10 14:00:52.367-07,t,2.0,f,pole,NaN,POINT (-122.32070 47.66103),238,img/seattle_with_icons/label_id_4.jpg
2,5,Devon Snyder,5,480,University District,SurfaceProblem,47.661060,-122.320709,OH4yKYW98UY1aMK_b68Etw,2022-05-10 14:01:21.608-07,t,2.0,f,bumpy,NaN,POINT (-122.32071 47.66106),94,img/seattle_with_icons/label_id_5.jpg
4,9,Devon Snyder,9,480,University District,Obstacle,47.661098,-122.320663,O0VZ-MF6Nrmjhkb2nCRhCw,2022-05-10 14:02:29.636-07,t,3.0,f,sign,temporary due to construction,POINT (-122.32066 47.66110),237,img/seattle_with_icons/label_id_9.jpg
18,30,Devon Snyder,30,483,University District,Obstacle,47.661419,-122.320488,iI69DtsYap_Fjiz-RZ4i1A,2022-05-10 14:13:05.844-07,f,2.0,f,pole,NaN,POINT (-122.32049 47.66142),113,img/seattle_with_icons/label_id_30.jpg
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2005,2493,mikey,2493,810,Windermere,SurfaceProblem,47.672043,-122.267929,Z6BJD4bSdDiMxwMdls5oNw,2022-07-06 14:12:40.653-07,NaN,1.0,f,uneven/slanted,NaN,POINT (-122.26793 47.67204),6,img/seattle_with_icons/label_id_2493.jpg
2006,2494,mikey,2494,716,Windermere,Obstacle,47.671928,-122.268097,sOyfUL3XT8ETE-9XfeaXvQ,2022-07-06 14:14:34.691-07,NaN,1.0,f,vegetation,NaN,POINT (-122.26810 47.67193),5,img/seattle_with_icons/label_id_2494.jpg
2008,2496,mikey,2496,716,Windermere,SurfaceProblem,47.671883,-122.268181,xbqjQfJSHA1O90JnBHQJGw,2022-07-06 14:14:55.957-07,NaN,1.0,f,cracks,NaN,POINT (-122.26818 47.67188),4,img/seattle_with_icons/label_id_2496.jpg
2016,2534,Devon Snyder,2534,479,University District,SurfaceProblem,47.659721,-122.319923,SRZ-Py452sb8rEJfkHzHWw,2022-11-09 08:08:26.318-08,NaN,2.0,f,"cracks,uneven/slanted",NaN,POINT (-122.31992 47.65972),2,img/seattle_with_icons/label_id_2534.jpg


In [29]:
#get center location of the geodf
seattle.dissolve().centroid

0    POINT (-122.33162 47.61003)
dtype: geometry

In [30]:
seattle_problem

,Unnamed: 0,username,label_id,street_edge_id,neighborhood,label_type,lat,lng,gsv_panorama_id,time_created,correct,severity,temporary,tag_list,description,geometry,cluster_id,img_url
0,3,Devon Snyder,3,480,University District,Obstacle,47.661041,-122.320709,-L8Sgb0ozIqqanCBfAZjYQ,2022-05-10 14:00:36.787-07,t,3.0,f,pole,NaN,POINT (-122.32071 47.66104),240,img/seattle_with_icons/label_id_3.jpg
1,4,Devon Snyder,4,480,University District,Obstacle,47.661026,-122.320702,-L8Sgb0ozIqqanCBfAZjYQ,2022-05-10 14:00:52.367-07,t,2.0,f,pole,NaN,POINT (-122.32070 47.66103),238,img/seattle_with_icons/label_id_4.jpg
2,5,Devon Snyder,5,480,University District,SurfaceProblem,47.661060,-122.320709,OH4yKYW98UY1aMK_b68Etw,2022-05-10 14:01:21.608-07,t,2.0,f,bumpy,NaN,POINT (-122.32071 47.66106),94,img/seattle_with_icons/label_id_5.jpg
4,9,Devon Snyder,9,480,University District,Obstacle,47.661098,-122.320663,O0VZ-MF6Nrmjhkb2nCRhCw,2022-05-10 14:02:29.636-07,t,3.0,f,sign,temporary due to construction,POINT (-122.32066 47.66110),237,img/seattle_with_icons/label_id_9.jpg
18,30,Devon Snyder,30,483,University District,Obstacle,47.661419,-122.320488,iI69DtsYap_Fjiz-RZ4i1A,2022-05-10 14:13:05.844-07,f,2.0,f,pole,NaN,POINT (-122.32049 47.66142),113,img/seattle_with_icons/label_id_30.jpg
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2005,2493,mikey,2493,810,Windermere,SurfaceProblem,47.672043,-122.267929,Z6BJD4bSdDiMxwMdls5oNw,2022-07-06 14:12:40.653-07,NaN,1.0,f,uneven/slanted,NaN,POINT (-122.26793 47.67204),6,img/seattle_with_icons/label_id_2493.jpg
2006,2494,mikey,2494,716,Windermere,Obstacle,47.671928,-122.268097,sOyfUL3XT8ETE-9XfeaXvQ,2022-07-06 14:14:34.691-07,NaN,1.0,f,vegetation,NaN,POINT (-122.26810 47.67193),5,img/seattle_with_icons/label_id_2494.jpg
2008,2496,mikey,2496,716,Windermere,SurfaceProblem,47.671883,-122.268181,xbqjQfJSHA1O90JnBHQJGw,2022-07-06 14:14:55.957-07,NaN,1.0,f,cracks,NaN,POINT (-122.26818 47.67188),4,img/seattle_with_icons/label_id_2496.jpg
2016,2534,Devon Snyder,2534,479,University District,SurfaceProblem,47.659721,-122.319923,SRZ-Py452sb8rEJfkHzHWw,2022-11-09 08:08:26.318-08,NaN,2.0,f,"cracks,uneven/slanted",NaN,POINT (-122.31992 47.65972),2,img/seattle_with_icons/label_id_2534.jpg


In [31]:
append_img_url(chicago_problem, chicago_folder)

,username,label_id,street_edge_id,neighborhood,label_type,lat,lng,gsv_panorama_id,time_created,correct,severity,temporary,tag_list,description,Unnamed: 14,Unnamed: 15,Unnamed: 16,geometry,cluster_id,clustered,img_url
80,Devon Snyder,99,447,Skokie,SurfaceProblem,42.022999,-87.728149,2vMz67hXm2ZGNcKHbyoR8g,2022-05-10 14:59:05.467-07,t,1.0,f,grass,NaN,NaN,NaN,NaN,POINT (-87.72815 42.02300),65,False,img/chicago_with_icons/label_id_99.jpg
81,Devon Snyder,100,447,Skokie,SurfaceProblem,42.023033,-87.728149,2vMz67hXm2ZGNcKHbyoR8g,2022-05-10 14:59:10.286-07,t,1.0,f,NaN,NaN,NaN,NaN,NaN,POINT (-87.72815 42.02303),67,False,img/chicago_with_icons/label_id_100.jpg
83,Devon Snyder,103,447,Skokie,SurfaceProblem,42.023464,-87.728004,BMhU3MUjwcgFnO2ax-6TxQ,2022-05-10 15:01:27.068-07,t,2.0,f,cracks,grass,NaN,NaN,NaN,POINT (-87.72800 42.02346),94,False,img/chicago_with_icons/label_id_103.jpg
85,Devon Snyder,105,447,Skokie,SurfaceProblem,42.023663,-87.728111,spnkMkInfnfp6Se2o7-t9Q,2022-05-10 15:02:43.061-07,t,1.0,f,grass,cracks,NaN,NaN,NaN,POINT (-87.72811 42.02366),92,False,img/chicago_with_icons/label_id_105.jpg
86,Devon Snyder,106,447,Skokie,SurfaceProblem,42.024178,-87.728104,-YaXYeIUgy5a5bYJcFDMQQ,2022-05-10 15:03:40.542-07,t,1.0,f,cracks,grass,NaN,NaN,NaN,POINT (-87.72810 42.02418),1,True,img/chicago_with_icons/label_id_106.jpg
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1898,mikey,2368,437,Skokie,SurfaceProblem,42.023743,-87.728401,UUSs_oAVBV5cqnoLKsUnlQ,2022-07-05 12:25:19.838-07,NaN,1.0,f,NaN,NaN,NaN,NaN,NaN,POINT (-87.72840 42.02374),7,True,img/chicago_with_icons/label_id_2368.jpg
1899,mikey,2369,437,Skokie,SurfaceProblem,42.023582,-87.728416,Ue0KqJfGftuJWhdBCneCkQ,2022-07-05 12:25:31.351-07,NaN,1.0,f,uneven/slanted,NaN,NaN,NaN,NaN,POINT (-87.72842 42.02358),6,True,img/chicago_with_icons/label_id_2369.jpg
1908,mikey,2378,404,Skokie,SurfaceProblem,42.025799,-87.728088,jfd17OkETDGKKw-I1zK6Jw,2022-07-05 12:38:58.8-07,NaN,1.0,f,bumpy,NaN,NaN,NaN,NaN,POINT (-87.72809 42.02580),5,False,img/chicago_with_icons/label_id_2378.jpg
1910,mikey,2380,404,Skokie,Obstacle,42.025677,-87.728088,Y-iz2Xxh7q_fAX_UH6mmVA,2022-07-05 12:39:07.658-07,NaN,1.0,t,NaN,NaN,NaN,NaN,NaN,POINT (-87.72809 42.02568),3,False,img/chicago_with_icons/label_id_2380.jpg


In [31]:
chicago_problem

,Unnamed: 0,username,label_id,street_edge_id,neighborhood,label_type,lat,lng,gsv_panorama_id,time_created,correct,severity,temporary,tag_list,description,geometry,cluster_id,img_url
80,99,Devon Snyder,99,447,Skokie,SurfaceProblem,42.022999,-87.728149,2vMz67hXm2ZGNcKHbyoR8g,2022-05-10 14:59:05.467-07,t,1.0,f,grass,NaN,POINT (-87.72815 42.02300),63,img/chicago_with_icons/label_id_99.jpg
81,100,Devon Snyder,100,447,Skokie,SurfaceProblem,42.023033,-87.728149,2vMz67hXm2ZGNcKHbyoR8g,2022-05-10 14:59:10.286-07,t,1.0,f,NaN,NaN,POINT (-87.72815 42.02303),67,img/chicago_with_icons/label_id_100.jpg
84,105,Devon Snyder,105,447,Skokie,SurfaceProblem,42.023663,-87.728111,spnkMkInfnfp6Se2o7-t9Q,2022-05-10 15:02:43.061-07,t,1.0,f,"grass,cracks",NaN,POINT (-87.72811 42.02366),88,img/chicago_with_icons/label_id_105.jpg
85,106,Devon Snyder,106,447,Skokie,SurfaceProblem,42.024178,-87.728104,-YaXYeIUgy5a5bYJcFDMQQ,2022-05-10 15:03:40.542-07,t,1.0,f,"grass,cracks",NaN,POINT (-87.72810 42.02418),1,img/chicago_with_icons/label_id_106.jpg
91,113,Devon Snyder,113,437,Skokie,Obstacle,42.024498,-87.728394,p_-iwPEGdz3zFZzojysq_Q,2022-05-10 15:08:14.409-07,t,4.0,t,construction,NaN,POINT (-87.72839 42.02450),8,img/chicago_with_icons/label_id_113.jpg
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1883,2368,mikey,2368,437,Skokie,SurfaceProblem,42.023743,-87.728401,UUSs_oAVBV5cqnoLKsUnlQ,2022-07-05 12:25:19.838-07,NaN,1.0,f,NaN,NaN,POINT (-87.72840 42.02374),7,img/chicago_with_icons/label_id_2368.jpg
1884,2369,mikey,2369,437,Skokie,SurfaceProblem,42.023582,-87.728416,Ue0KqJfGftuJWhdBCneCkQ,2022-07-05 12:25:31.351-07,NaN,1.0,f,uneven/slanted,NaN,POINT (-87.72842 42.02358),6,img/chicago_with_icons/label_id_2369.jpg
1893,2378,mikey,2378,404,Skokie,SurfaceProblem,42.025799,-87.728088,jfd17OkETDGKKw-I1zK6Jw,2022-07-05 12:38:58.8-07,NaN,1.0,f,bumpy,NaN,POINT (-87.72809 42.02580),5,img/chicago_with_icons/label_id_2378.jpg
1895,2380,mikey,2380,404,Skokie,Obstacle,42.025677,-87.728088,Y-iz2Xxh7q_fAX_UH6mmVA,2022-07-05 12:39:07.658-07,NaN,1.0,t,NaN,NaN,POINT (-87.72809 42.02568),3,img/chicago_with_icons/label_id_2380.jpg


In [42]:
seattle_problem["clustered"] = seattle_problem.duplicated(subset="cluster_id", keep=False)

In [43]:
seattle_problem.to_file('/Users/chuli/Documents/GitHub/ps-validation-study.github.io/data/seattle_problem.json', driver="GeoJSON") 

In [44]:
seattle_curb_ramp["clustered"] = seattle_curb_ramp.duplicated(subset="cluster_id", keep=False)

In [45]:
seattle_curb_ramp.to_file('/Users/chuli/Documents/GitHub/ps-validation-study.github.io/data/seattle_curb_ramp.json', driver="GeoJSON") 

In [46]:
chicago_problem["clustered"] = chicago_problem.duplicated(subset="cluster_id", keep=False)

In [47]:
chicago_problem.to_file('/Users/chuli/Documents/GitHub/ps-validation-study.github.io/data/chicago_problem.json', driver="GeoJSON") 

In [48]:
chicago_curb_ramp["clustered"] = chicago_curb_ramp.duplicated(subset="cluster_id", keep=False)

In [49]:
chicago_curb_ramp.to_file('/Users/chuli/Documents/GitHub/ps-validation-study.github.io/data/chicago_curb_ramp.json', driver="GeoJSON") 